Copyright 2024 - Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Gemini Pro - Using Vertex AI Search as a grounding service.

* This notebook explains how to use grounding service in Gemini Pro.
* Refer to https://cloud.google.com/vertex-ai/generative-ai/docs/grounding/overview
* Architecture for Grounding with Google Search
 * [Grounding with Google search](https://cloud.google.com/static/vertex-ai/generative-ai/docs/images/gemini/overview-grounding.png)

* Using Vertex AI Search :
  * https://cloud.google.com/vertex-ai/generative-ai/docs/grounding/overview#ground-private

# Configuration
## Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest


In [ ]:
%pip install --upgrade --quiet google-cloud-aiplatform

In [ ]:
from IPython.display import display, Markdown

## Authentication to access to the GCP & Google drive

* Use OAuth to access the GCP environment.
 * Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko

In [ ]:
#  For only colab to authenticate to get an access to the GCP.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

    # Mount to the google drive to access the .ipynb files in the repository.
    from google.colab import drive
    drive.mount('/content/drive')

* **굵은 텍스트** Mount to the google drive to access the .ipynb files in the repository.

# Execute the example
## Set the environment on GCP Project
* Configure project information
  * Model name : LLM model name : https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models
  * Project Id : prodect id in GCP
  * Region : region name in GCP

In [ ]:
MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.
* Vertex AI initialization : aiplatform.init(..)
  * https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

In [ ]:
import vertexai
from vertexai.generative_models import GenerationConfig, GenerativeModel, Part, Tool
import vertexai.generative_models as generative_models

# Grounding service is still in preview.
from vertexai.preview.generative_models import grounding

# Initalizate the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION)

# Access to the generative model.
model = GenerativeModel(MODEL_NAME)

## Prompt sent to LLM

In [ ]:
prompt ="민간 부문의 양자 정보 통신 기술 개발 지원에 대해서 설명해주세요."

## Response without grounding services.
This response is information from LLM without grounding services to enrich contexts sent to LLM.

In [ ]:
response = model.generate_content(prompt)
display(Markdown(response.text))

##Configuration Vertex AI Search

In [ ]:
DATA_STORE_PROJECT_ID = PROJECT_ID
DATA_STORE_REGION = "global"

# Note : Only Digital parser datastore is possible to us grounding service as of Aug 2024

DATA_STORE_ID = "hackathon-ds_1709838247983"

## Execute request to LLM with grounding servcies supported by Vertex AI search.

In [ ]:

# datastore = f"projects/{DATA_STORE_PROJECT_ID}/locations/{DATA_STORE_REGION}/collections/default_collection/dataStores/{DATA_STORE_ID}"

data_store_path = f"projects/{DATA_STORE_PROJECT_ID}/locations/{DATA_STORE_REGION}/collections/default_collection/dataStores/{DATA_STORE_ID}"


tool = Tool.from_retrieval(
    grounding.Retrieval(grounding.VertexAISearch(datastore=data_store_path))
)

response = model.generate_content(prompt, tools=[tool])

display(Markdown(response.text))

## Fact finding

In [ ]:
# from IPython.display import Image
# image_path = "/content/drive/MyDrive/projects/google_gen_ai_sample/contents/images/"
# image_file = "grounding_service_vertexai.png"
# Image(f"{image_path}{image_file}", width=700, height=300)

